# 4.3.	Violações dos padrões de qualidade do ar no Brasil - CAMILO

Tabela 10 – Ranqueamento das cinco estações com o maior número de violações dos Padrões de Qualidade do Ar de O 3, MP10, MP2,5 e SO2 no ano de 2023. 
    TENTAR FAZER UMA TABELA ITERATIVA ONDE O USUÁRIO POSSA FILTRAR AS ESTAÇÕES

    TENTAR FAZER UM DROPDOWN MENU PARA ESCOLHER QUAL SÉRIE TEMPORAL VISUALIZAR NO LIVRO. Colocar padrões da CONAMA. 

<!-- Dependencies -->
<script src="https://cdn.plot.ly/plotly-2.26.1.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/papaparse@5.4.1/papaparse.min.js"></script>

<div>
  <label>Station:</label>
  <select id="stationSelect"></select>

  <label>Pollutant:</label>
  <select id="pollutantSelect"></select>

  <label>Average Time:</label>
  <select id="aveTimeSelect"></select>

  <label>Start date:</label>
  <input type="datetime-local" id="startDate">

  <label>End date:</label>
  <input type="datetime-local" id="endDate">

  <button id="loadDataBtn">Load Data</button>
  <button id="plotBtn" disabled>Plot Data</button>
  <button id="downloadBtn" disabled>Download CSV</button>
</div>

<div id="plotDiv" style="width:100%; height:600px;"></div>
<div id="statsTable" style="margin-top:20px;"></div>


<script>
const DATA_FOLDER = 'https://lcqar.ufsc.br/files/2023';
const METADATA_CSV = 'https://lcqar.ufsc.br/files/2023/Monitoramento_QAr_BR_teste.csv';
const FASES_CSV = '../_static/fases_CONAMA506.csv';

let metadata = [];
let fasesData = [];
let stationPollutants = {};
let currentData = [];
let filteredData = [];

// Promise wrapper for Papa.parse
function parseCSV(url) {
  return new Promise((resolve, reject) => {
    Papa.parse(url, {
      download: true,
      header: true,
      skipEmptyLines: true,
      complete: res => resolve(res.data),
      error: err => reject(err)
    });
  });
}

// Init: load CSVs
Promise.all([parseCSV(METADATA_CSV), parseCSV(FASES_CSV)])
  .then(([metaRows, fasesRows]) => {
    metadata = metaRows.filter(r => r.ID_MMA && r.POLUENTE);
    fasesData = fasesRows;

    buildStationPollutantsMap();
    populatePollutantSelect();
    populateStationSelect();
    updateAveTimeDropdown();

    const polSelect = document.getElementById('pollutantSelect');
    polSelect.addEventListener('change', () => {
      populateStationSelect();
      updateAveTimeDropdown();
      document.getElementById('plotBtn').disabled = true;
      document.getElementById('downloadBtn').disabled = true;
    });

    document.getElementById('loadDataBtn').addEventListener('click', loadStationData);
    document.getElementById('plotBtn').addEventListener('click', doPlot);
    document.getElementById('downloadBtn').addEventListener('click', downloadFilteredCSV);
  })
  .catch(err => {
    console.error('Erro ao carregar CSVs:', err);
    alert('Erro ao carregar CSVs (veja console).');
  });

function buildStationPollutantsMap() {
  stationPollutants = {};
  metadata.forEach(row => {
    const id = (row.ID_MMA || '').trim();
    const pol = (row.POLUENTE || '').trim();
    if (!id || !pol) return;
    if (!stationPollutants[id]) stationPollutants[id] = new Set();
    stationPollutants[id].add(pol);
  });
  for (const st in stationPollutants) {
    stationPollutants[st] = Array.from(stationPollutants[st]).sort();
  }
}

function populatePollutantSelect() {
  const polSelect = document.getElementById('pollutantSelect');
  polSelect.innerHTML = '';
  const polSet = new Set(metadata.map(r => (r.POLUENTE || '').trim()).filter(Boolean));
  Array.from(polSet).sort().forEach(pol => {
    const opt = document.createElement('option');
    opt.value = pol;
    opt.textContent = pol;
    polSelect.appendChild(opt);
  });
  if (polSelect.options.length) polSelect.selectedIndex = 0;
}

function populateStationSelect() {
  const pol = (document.getElementById('pollutantSelect').value || '').trim();
  const stSelect = document.getElementById('stationSelect');
  stSelect.innerHTML = '';
  if (!pol) return;

  const uniqueStations = Array.from(
    new Set(metadata.filter(r => (r.POLUENTE || '').trim() === pol).map(r => (r.ID_MMA || '').trim()))
  ).filter(Boolean).sort();
  uniqueStations.forEach(st => {
    const opt = document.createElement('option');
    opt.value = st;
    opt.textContent = st;
    stSelect.appendChild(opt);
  });
  if (stSelect.options.length) stSelect.selectedIndex = 0;
}

function updateAveTimeDropdown() {
  const pollutant = (document.getElementById("pollutantSelect").value || '').trim();
  const aveTimeSelect = document.getElementById("aveTimeSelect");
  aveTimeSelect.innerHTML = "";

  const aveTimes = [
    ...new Set(
      fasesData
        .filter(r => ((r.pollutant || '').trim()).startsWith(pollutant))
        .map(r => (r.ave_time || '').trim())
        .filter(Boolean)
    )
  ].sort();

  if (!aveTimes.length) {
    const opt = document.createElement('option');
    opt.value = '';
    opt.textContent = '-- none --';
    aveTimeSelect.appendChild(opt);
    return;
  }

  aveTimes.forEach(at => {
    const opt = document.createElement("option");
    opt.value = at;
    opt.textContent = at;
    aveTimeSelect.appendChild(opt);
  });
  aveTimeSelect.selectedIndex = 0;
}

function loadStationData() {
  const pollutant = (document.getElementById('pollutantSelect').value || '').trim();
  const station = (document.getElementById('stationSelect').value || '').trim();
  if (!pollutant || !station) {
    alert('Selecione estação e poluente antes de carregar dados.');
    return;
  }
  const fileUrl = `${DATA_FOLDER}/${encodeURIComponent(pollutant)}/${encodeURIComponent(station)}.csv`;
  Papa.parse(fileUrl, {
    download: true,
    header: true,
    skipEmptyLines: true,
    complete: res => {
      currentData = res.data || [];
      document.getElementById('plotBtn').disabled = currentData.length === 0;
      document.getElementById('downloadBtn').disabled = true;
      setDateLimits(currentData);
    },
    error: err => {
      console.error('Erro ao carregar arquivo:', err);
      document.getElementById('plotBtn').disabled = true;
      document.getElementById('downloadBtn').disabled = true;
    }
  });
}

function setDateLimits(data) {
  const dates = data.map(row => {
    const y = parseInt(row.ANO, 10);
    const m = parseInt(row.MES, 10);
    const d = parseInt(row.DIA, 10);
    const h = parseInt(row.HORA, 10) || 0;
    if ([y,m,d].some(v => Number.isNaN(v))) return null;
    return new Date(y, m - 1, d, h);
  }).filter(d => d instanceof Date && !isNaN(d));
  if (!dates.length) return;
  const minDate = new Date(Math.min(...dates));
  const maxDate = new Date(Math.max(...dates));
  const fmt = d => `${d.getFullYear()}-${String(d.getMonth()+1).padStart(2,'0')}-${String(d.getDate()).padStart(2,'0')}T${String(d.getHours()).padStart(2,'0')}:${String(d.getMinutes()).padStart(2,'0')}`;
  document.getElementById('startDate').value = fmt(minDate);
  document.getElementById('endDate').value = fmt(maxDate);
}

function doPlot() {
  if (!currentData.length) return;

  const pollutant = (document.getElementById('pollutantSelect').value || '').trim();
  const ave_time = (document.getElementById("aveTimeSelect").value || '').trim().toLowerCase();
  const startDateVal = document.getElementById('startDate').value;
  const endDateVal = document.getElementById('endDate').value;
  const startDate = startDateVal ? new Date(startDateVal) : null;
  const endDate = endDateVal ? new Date(endDateVal) : null;

  // Map CSV to datetime + numeric
  filteredData = currentData.map(row => {
    const y = parseInt(row.ANO, 10);
    const m = parseInt(row.MES, 10);
    const d = parseInt(row.DIA, 10);
    const h = parseInt(row.HORA, 10) || 0;
    const dt = (Number.isNaN(y) || Number.isNaN(m) || Number.isNaN(d)) ? null : new Date(y, m - 1, d, h);

    let val = null;
    if (row.VALOR !== undefined && row.VALOR !== null) {
      const num = parseFloat(String(row.VALOR).replace(',', '.').trim());
      if (!Number.isNaN(num) && num >= 0) val = num;
    }

    return { datetime: dt, valor: val };
  })
  .filter(r => r.datetime && (!startDate || r.datetime >= startDate) && (!endDate || r.datetime <= endDate))
  .sort((a, b) => a.datetime - b.datetime);


  // --- APPLY TIME AVERAGING ---
  if (ave_time.includes("24") || ave_time.includes("day") || ave_time === "d") {
    const grouped = {};
    filteredData.forEach(p => {
      if (p.valor != null) {
        const dayKey = p.datetime.toISOString().slice(0,10);
        if (!grouped[dayKey]) grouped[dayKey] = [];
        grouped[dayKey].push(p.valor);
      }
    });
    filteredData = Object.entries(grouped).map(([day, vals]) => ({
      datetime: new Date(day),
      valor: vals.reduce((a,b)=>a+b,0)/vals.length
    }));
  } else if (ave_time.includes("8")) {
    filteredData.sort((a,b)=>a.datetime-b.datetime);
    let rolling = [];
    const dailyMax = {};
    for (let i=0;i<filteredData.length;i++){
      rolling.push(filteredData[i]);
      if(rolling.length>8) rolling.shift();
      if(rolling.length>=6){
        const avg = rolling.reduce((s,r)=>s+r.valor,0)/rolling.length;
        const dayKey = filteredData[i].datetime.toISOString().slice(0,10);
        if(!dailyMax[dayKey] || avg>dailyMax[dayKey]) dailyMax[dayKey] = avg;
      }
    }
    filteredData = Object.entries(dailyMax).map(([day,maxVal])=>({datetime:new Date(day), valor:maxVal}));
  } else if (ave_time.includes("year") || ave_time.includes("ano") || ave_time.includes("anual") || ave_time.includes("annual")) {
    const grouped = {};
    filteredData.forEach(p=>{
      if(p.valor!=null){
        const yr = p.datetime.getFullYear();
        if(!grouped[yr]) grouped[yr]=[];
        grouped[yr].push(p.valor);
      }
    });
    filteredData = Object.entries(grouped).map(([yr,vals])=>({
      datetime:new Date(`${yr}-01-01`),
      valor:vals.reduce((a,b)=>a+b,0)/vals.length
    }));
  }

  // --- MAX VALUE AFTER AVERAGING ---
  const numericVals = filteredData.map(p=>p.valor).filter(v=>v!=null);
  const maxVal = numericVals.length ? Math.max(...numericVals)*1.1 : 10;

  // --- FILTER PHASES ---
  const fasesForPol = fasesData.filter(r=>{
    const pol = (r.pollutant||'').trim();
    const at = (r.ave_time||'').trim();
    return pol.startsWith(pollutant) && (!ave_time || at===ave_time);
  });

  const plotShapes = fasesForPol.map(r=>{
    let y0 = parseFloat(String(r.y0||'').replace(',','.'));
    let y1 = parseFloat(String(r.y1||'').replace(',','.'));

    // Apply PI-1 only for non-annual averages
    if (!(ave_time.includes("year") || ave_time.includes("ano") || ave_time.includes("anual") || ave_time.includes("annual"))) {
        if (r.phase==='PI-1') y1 = maxVal;
    }

    if(Number.isNaN(y0)||Number.isNaN(y1)) return null;
    return {type:'rect', xref:'paper', x0:0, x1:1, y0, y1, fillcolor:r.color||'rgba(200,200,200,0.3)', opacity:0.3, line:{width:0}, layer:'below'};
  }).filter(Boolean);

  const plotData = [];

  if (ave_time.includes("year") || ave_time.includes("ano") || ave_time.includes("anual") || ave_time.includes("annual")) {
      // --- ANNUAL BAR PLOT ---
      const years = filteredData.map(p => p.datetime);
      const values = filteredData.map(p => p.valor);

      plotData.push({
          type: 'bar',
          x: years,
          y: values,
          marker: { color: 'steelblue', opacity: 0.5 },
          width: 90*24*3600*1000, // ~3 months in ms, small width
          name: 'Annual Average'
      });

      var layout = {
          xaxis: {
              title: 'Ano',
              tickformat: '%Y',
              dtick: "M12"
          },
          yaxis: { title: 'Concentração', range: [0, Math.max(...values)*1.1], rangemode: 'nonnegative' },
          shapes: plotShapes,
          title: `Série anual - ${pollutant}`,
          hovermode: 'x unified',
          plot_bgcolor: 'rgba(0.9,0.9,0.9,0.3)'
      };
  } else {
        plotData.push({
          type: 'scatter',
          mode: 'lines',
          x: filteredData.map(p => p.datetime),
          y: filteredData.map(p => p.valor), // contém null onde não há dado
          line: { color: 'steelblue', width: 1.5 },
          connectgaps: false,   // <- ESSENCIAL
          name: pollutant
        });


      var layout = {
          title:`Série temporal - ${pollutant}`,
          hovermode:'x unified',
          plot_bgcolor:'rgba(0.9,0.9,0.9,0.3)',
          yaxis:{title:'Concentração', range:[0,maxVal], rangemode:'nonnegative'},
          xaxis:{title:'Data/Hora'},
          shapes: plotShapes
      };
  }

  Plotly.newPlot('plotDiv', plotData, layout, {responsive:true});
  document.getElementById('downloadBtn').disabled = filteredData.length===0;
   // --- BUILD EXCEEDANCE TABLE ---
  buildExceedanceTable(fasesForPol, numericVals);
}

function buildExceedanceTable(fasesForPol, numericVals) {
  const container = document.getElementById('statsTable');
  container.innerHTML = ""; // reset

  if (!numericVals.length || !fasesForPol.length) {
    container.innerHTML = "<p><i>Sem dados ou padrões para calcular excedências.</i></p>";
    return;
  }

  let html = "<table border='1' cellspacing='0' cellpadding='4' style='border-collapse:collapse; width:100%; text-align:center'>";
  html += "<thead><tr style='background:#f0f0f0'><th>Padrão</th><th>Limite (µg/m³)</th><th>Excedências</th><th>% Excedência</th></tr></thead><tbody>";

  const n = numericVals.length;

  fasesForPol.forEach(r => {
    let y1 = parseFloat(String(r.y1||'').replace(',','.'));
    if (isNaN(y1)) return;

    const exceedCount = numericVals.filter(v => v > y1).length;
    const perc = ((exceedCount / n) * 100).toFixed(1);

    html += `<tr style="background:${r.color || 'transparent'}20">
      <td>${r.phase || '-'}</td>
      <td>${y1}</td>
      <td>${exceedCount}</td>
      <td>${perc}%</td>
    </tr>`;
  });

  html += "</tbody></table>";
  container.innerHTML = html;
}



function downloadFilteredCSV(){
  if(!filteredData.length){alert('Nenhum dado filtrado para baixar.'); return;}
  const csvString = Papa.unparse(filteredData.map(r=>({DataHora:r.datetime?r.datetime.toISOString():'', Valor:r.valor})));
  const blob = new Blob([csvString], {type:'text/csv;charset=utf-8;'});
  const url = URL.createObjectURL(blob);
  const pollutant = (document.getElementById('pollutantSelect').value||'').trim();
  const station = (document.getElementById('stationSelect').value||'').trim();
  const filename = `${station}_${pollutant}_filtrado.csv`;
  const link=document.createElement("a"); link.href=url; link.download=filename; document.body.appendChild(link); link.click(); document.body.removeChild(link);
}
</script>

